In [ ]:
# the following code allows you to import function from
# jupyter notebook in other directory
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import functools



In [ ]:
# ogw_exp computes the open gromov witten invariant of (CP^n, RP^n)
# Input:
# n - dimension of the real projective space RP^n
# A - list of cohomology classes represented by integers, ranging from 1...n, and -1 representing \Gamma_{square}
# b - beta, a non-negative integer
# k - number of boundary marked points, a non-negative integer
@functools.lru_cache(maxsize=None)
def ogw_invariant(n, b, k, A : ClassList):
    l = len(A)
    #A.sort()    # asceding order
    #A.reverse() # to descending order

    #for c_d in A:
    #    if c_d > n:
    #        return 0

    if A.invalid_class_list:
        return 0

    # wall-crossing theorem (Elad)
    if l > 0 and A[0] == -1:
        assert False
        return -ogw_invariant(n, b, k+1, A[:-1])

    #degree axiom (Elad)
    if 2*sum(A) != (n - 3 + (n+1)*b + k + 2*l - k*n):  
        # The maslov index is twice the chern class n+1 for holomorphic spheres. 
        # Using the invoulotion we can deduce that its half of this for disks thus mu(b)=b*(n+1) 
        # p 595 mcduff 
        #ogw_d[str((n, b, k, A)) + ", degree axiom"] = 0
        return 0

    # other
    if k == 0 and is_even(b):
        return 0

    # zero axiom
    if b == 0:
        if k == 1 and l == 1 and A[0] == 0:
            return -1
        else:
            return 0

    # unit axiom (May)
    # handle all beta = 0 cases
    if l > 0 and A[0] == 0:
        if b == 0 and k == 1 and l == 1:
            return -1
        else:
            return 0

    # divisor axiom 
    if l > 0 and A[0] == 1:
        A[1] -= 1
        return (b/2)*ogw_invariant(n, b, k, A)

    # handle beta = 1, l <= 1 cases
    """
    if b == 1 and k == 2 and A == []:
        return 2
    if b == 1 and len(A) == 1 and k == 1:
        return 0

    if b == 1 and len(A) == 1 and k == 0:
        return (-1)^((n-1)/2)
        """
    if b == 1 and k == 0 and A == [n]:
        return (-1)^((n+3)/2)

    if b == 1 and k == 1 and A == [(n+1)/2]:
        return 0

    if b == 1 and k == 2 and A == [0]*n:
        return 2


    if l >= 2:
        cor = ogw_invariant(n, b, k, [A[0]-1, A[1]+1] + A[2:])
        col = 0
        ool = 0
        oor = 0

        for t in A[2:].bipart():
            I1 = t[0]
            I2 = t[1]

            for b1 in range(1, b):
                b_ = 0
                if (b - b1) % 2 == 0:
                    b_ = (b - b1) / 2
                else:
                    continue

                for i in range(0, n+1):
                    cor += bipart_coeff(A, I1) \
                        * (-1)^(b_*(n+1)/2) *  GW_invariant(n, b_, ([1, A[1], i] + I1).decompress()) \
                        *  ogw_invariant(n, b1, k, [n-i, A[0]-1] + I2) 

                        # I1 = [1,1,1,1,2,2,2]
                        # I1.c_list = [0, 4, 3, 0]
                        # [1, 2, 3] + I1
                        # I1 + [1, 2, 3] = [0, 5, 4, 1]


            for b1 in range(1, b):
                b_ = 0
                if (b - b1) % 2 == 0:
                    b_ = (b - b1) / 2
                else:
                    continue

                for i in range(0, n+1):
                    col += bipart_coeff(A, I1) \
                        * (-1)^((n+1)*b_/2)* GW_invariant(n, b_, ([1, A[0] - 1, i] + I1).decompress()) \
                        * ogw_invariant(n, b1, k, [n-i, A[1]] + I2)

            
            for b1 in range(1, b):
                b2 = b - b1

                for k1 in range(0, k+1):
                    k2 = k - k1
                    ool += bipart_coeff(A, I1) \
                        * binomial(k, k1)*ogw_invariant(n, b1, k1, [1, A[0]-1] + I1) \
                        * ogw_invariant(n, b2, k2 + 1, [A[1]] + I2)

            
            for b1 in range(1, b):
                b2 = b - b1

                for k1 in range(0, k+1):
                    k2 = k - k1

                    oor += bipart_coeff(A, I1) \
                        * binomial(k, k1) * ogw_invariant(n, b1, k1, [1, A[1]] + I1) \
                        * ogw_invariant(n, b2, k2 + 1, [A[0] - 1] + I2)

        return cor - col - oor + ool

    if k >= 2:
        co = 0
        ool = 0
        oor = 0

        for t in A.bipart():
            I1 = t[0]
            I2 = t[1]

            for b1 in range(1, b+1):
                b_ = 0
                if (b + 1 - b1) % 2 == 0:
                    b_ = (b + 1 - b1) / 2
                else:
                    continue

                for i in range(0, n+1):
                    co += bipart_coeff(A, I1) \
                        * (-1)^((n+1)*b_/2)*GW_invariant(n, b_, ([1, n, i] + I1).decompress()) \
                        * ogw_invariant(n, b1, k-1, [n-i] + I2)                   
            
            for b1 in range(2, b+1):
                b2 = b + 1 - b1

                for k1 in range(0, (k - 2)+1):
                    k2 = k - 2 - k1
                    ool += bipart_coeff(A, I1) \
                        * binomial(k-2, k1)*ogw_invariant(n, b1, k1, [1, n] + I1)\
                        * ogw_invariant(n, b2, k2+2, I2)

            for b1 in range(1, b+1):
                b2 = b + 1 - b1

                for k1 in range(0, (k - 2)+1):
                    k2 = k - 2 - k1
                    oor += bipart_coeff(A, I1) \
                        * binomial(k-2, k1) * ogw_invariant(n, b1, k1 + 1, [1] + I1) \
                        * ogw_invariant(n, b2, k2+1, [n]+I2)

        return (-1)^((n+3)/2)*2*(co - ool + oor)

    print("unexpected situation")
    print("n: ", n, ", b: ", b, ", k: ", k, " A: ", A)
    return 0
